This is for tokenizing the dataset
and data_collator puts the batches

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer
trainer= Trainer(model,
                 training_args,
                 train_dataset=tokenized_datasets["train"],
                 eval_dataset=tokenized_datasets["validation"],
                 data_collator=data_collator,
                 processing_class=tokenizer)
trainer.train()

In [ ]:
from transformers import Trainer
predictions =trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [8]:
# tokenized_datasets=tokenized_datasets.remove_columns(["sentence1","sentence2","idx"])
# tokenized_datasets=tokenized_datasets.rename_column("label","labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [10]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(tokenized_datasets["train"],shuffle=True,batch_size=8,collate_fn=data_collator)
eval_dataloader=DataLoader(tokenized_datasets["validation"],batch_size=8,collate_fn=data_collator)

In [14]:
for batch in train_dataloader:
  break
{k:v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 63]),
 'token_type_ids': torch.Size([8, 63]),
 'attention_mask': torch.Size([8, 63])}

In [15]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
output=model(**batch)
print(output.loss,output.logits.shape)

tensor(0.6988, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [18]:
from torch.optim import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5)

In [20]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


In [21]:
import torch
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch ={k: v.to(device) for k,v in batch.items()}
    output=model(**batch)
    loss=output.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [30]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8333333333333334, 'f1': 0.8843537414965986}

for sst2 dataset

In [3]:
from datasets import load_dataset

raw_dataset = load_dataset("gimmaru/glue-sst2")
raw_dataset

README.md:   0%|          | 0.00/775 [00:00<?, ?B/s]

(…)-00000-of-00001-02c8af09f7a87704.parquet:   0%|          | 0.00/72.7k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
})

In [4]:
raw_dataset=raw_dataset["validation"].train_test_split(test_size=0.2)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 697
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 175
    })
})

In [6]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,DataCollatorWithPadding
checkpoint="bert-base-uncased"
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

def function(example):
  return tokenizer(example["sentence"])

tokenized_dataset=raw_dataset.map(function,batched=True)
tokenized_dataset
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
final_tokenized_dataset=tokenized_dataset.remove_columns(["sentence","idx"])
final_tokenized_dataset=final_tokenized_dataset.rename_column("label","labels")
final_tokenized_dataset["train"].column_names
final_tokenized_dataset.set_format("torch")

In [54]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(final_tokenized_dataset["train"],shuffle=True,batch_size=3,collate_fn=data_collator)
test_dataloader=DataLoader(final_tokenized_dataset["test"],shuffle=True,batch_size=3,collate_fn=data_collator)


In [32]:
for batch in train_dataloader:
  break
{k:v.shape for k,v in batch.items()}


{'labels': torch.Size([3]),
 'input_ids': torch.Size([3, 40]),
 'token_type_ids': torch.Size([3, 40]),
 'attention_mask': torch.Size([3, 40])}

In [34]:
output=model(**batch)
output

SequenceClassifierOutput(loss=tensor(0.5915, grad_fn=<NllLossBackward0>), logits=tensor([[-0.3389,  0.4010],
        [-0.4305,  0.4638],
        [-0.4730,  0.5704]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [36]:
from torch.optim import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5)

In [38]:
from transformers import get_scheduler
num_epochs=3
num_training_steps=num_epochs*len(train_dataloader)
lr_scheduler=get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps

)
print(num_training_steps)

699


In [39]:
import torch
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [41]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch={k: v.to(device) for k,v in batch.items()}
    output=model(**batch)
    loss=output.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    progress_bar.update(1)

  0%|          | 0/699 [00:00<?, ?it/s]

In [57]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.4685714285714286, 'f1': 0.0}

In [46]:
from transformers import AutoTokenizer,DataCollatorWithPadding
checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

def function(example):
  return tokenizer(example["sentence"])

tokenized_test=raw_dataset["test"].map(function,batched=True)

data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_test

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 175
})

In [51]:
# tokenized_test=tokenized_test.remove_columns(["sentence"])
# tokenized_test=tokenized_test.rename_column("label","labels")

final_tokenized_dataset.set_format("torch")
final_tokenized_dataset["test"].column_names

{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}

In [5]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [56]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00


In [3]:
!pip install --upgrade datasets fsspec huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.4
    Uninstalling huggingface-hub-0.31.4:
      Successfully uninstalled huggingface-hub-0.31.4
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c